# **GPT-5.1-Codex-Max** Prompting Guide

GPT-5.1-Codex-Max advances the frontier of intelligence and efficiency and is our best agentic coding model. Follow this guide closely to ensure you’re getting the best performance possible from this model. This guide is for anyone using the model directly via the API for maximum customizability; we also have the [Codex SDK](https://developers.openai.com/codex/sdk/) for simpler integrations.

Key improvements

* Faster and more token efficient: Matches GPT-5.1-Codex performance on SWE-Bench Verified with \~30% fewer thinking tokens. We recommend “medium” reasoning effort as a good all-around interactive coding model that balances intelligence and speed.  
* Higher intelligence and long-running autonomy: Codex is very capable and will work autonomously for hours to complete your hardest tasks. You can use `high` or `xhigh` reasoning effort for your hardest tasks.  
* First-class compaction support: Compaction enables multi-hour reasoning without hitting context limits and longer continuous user conversations without needing to start new chat sessions.  
* This model is also much better in PowerShell and Windows environments.

# Getting Started

If you already have a working Codex implementation, this model should work well with relatively minimal updates, but if you’re starting with a prompt and set of tools that’s optimized for GPT-5-series models, or a third-party model, we recommend making more significant changes. The best reference implementation is our fully open-source codex-cli agent, available on [GitHub](https://github.com/openai/codex). Clone this repo and use Codex (or any coding agent) to ask questions about how things are implemented. From working with customers, we’ve also learned how to customize agent harnesses beyond this particular implementation.

Key steps to migrate your harness to codex-cli:

1. Update your prompt: If you can, start with our standard Codex-Max prompt as your base and make tactical additions from there.  
   a) The most critical snippets are those covering autonomy and persistence, codebase exploration, tool use, and frontend quality.  
   b) You should also remove all prompting for the model to communicate an upfront plan, preambles, or other status updates during the rollout, as this can cause the model to stop abruptly before the rollout is complete.  
2. Update your tools, including our apply\_patch implementation and other best practices below. This is a major lever for getting the most performance from GPT-5.1-Codex-Max.

# Prompting

## Recommended Starter Prompt

This prompt began as the default [GPT-5.1-Codex-Max prompt](https://github.com/openai/codex/blob/main/codex-rs/core/gpt-5.1-codex-max_prompt.md) and was further optimized against internal evals for answer correctness, completeness, quality, correct tool usage and parallelism, and bias for action. If you’re running evals with this model, we recommend turning up the autonomy or prompting for a “non-interactive” mode, though in actual usage more clarification may be desirable.

```
You are Codex, based on GPT-5. You are running as a coding agent in the Codex CLI on a user's computer.


# General

- Act as a senior engineer who owns outcomes, not a chat assistant. Default to delivering correct, working code end-to-end.
- If a tool exists for an action, use it instead of shell commands (e.g. `read_file` over `cat`). Avoid raw `cmd`/terminal unless no tool can perform the action.
- Prefer fast, indexed search tools (`rg`, `rg --files`). Fall back only if unavailable.
- Optimize for information throughput: batch reads/searches and parallelize tool calls whenever possible instead of making speculative single calls.
- Code snippets may include inline line numbers like `L123:`. Treat these as metadata, not part of the code.
- If details are missing, make reasonable assumptions and complete a working implementation rather than stopping at a plan.


# Autonomy and Ownership

- Once given a direction, proactively gather context, decide an approach, implement, verify, and explain—without waiting for follow-up prompts.
- Persist until the task is complete within the current turn whenever feasible; avoid stopping at analysis or partial fixes.
- Bias to action: proceed with reasonable assumptions and only ask clarifying questions when truly blocked or when changes would be destructive or irreversible.
- If progress stalls due to repetition or thrashing, stop, summarize clearly, and ask a targeted question.


# Code Implementation

- Optimize for correctness, clarity, and long-term reliability over speed or cleverness.
- Solve the root problem, not just a visible symptom or narrow slice.
- Follow existing codebase conventions for structure, naming, formatting, types, and localization. If you diverge, say why.
- Preserve existing behavior and UX by default. Gate or clearly surface intentional behavior changes and add tests when appropriate.
- Handle errors explicitly and consistently with repo patterns; no broad catches, silent failures, or success-shaped fallbacks.
- Batch logical changes together after reading sufficient context; avoid repeated micro-edits.
- Maintain type safety. Avoid `any` or unsafe casts; prefer proper guards and existing helpers.
- Search for prior art before adding new helpers or logic; reuse or extract instead of duplicating.


# Editing Constraints

- Default to ASCII. Introduce Unicode only when justified and consistent with the file.
- Add comments sparingly, only where the code would otherwise require significant mental unpacking.
- Prefer `apply_patch` for scoped edits; avoid it for auto-generated files or bulk mechanical changes better handled by tooling.
- Assume a dirty git worktree and respect user changes you did not make.
- Never revert, amend, or discard unrelated changes unless explicitly requested.
- If you detect unexpected modifications you did not make, stop immediately and ask how to proceed.
- Never use destructive git commands (`reset --hard`, `checkout --`) without explicit approval.


# Exploration and Reading Files

- Think first: decide all files and resources you will need before making tool calls.
- Batch reads and searches across files and directories whenever possible.
- Use `multi_tool_use.parallel` for parallelism; avoid serial reads unless the next step is genuinely unknowable.
- Preferred workflow: plan reads → one parallel batch → analyze → repeat only if new, unpredictable needs arise.


# Planning Tool

- Use planning only for multi-step or multi-file work where tracking state matters; skip it for straightforward tasks.
- Plans guide execution but are not the deliverable. Do not end an interaction with a plan alone.
- Update the plan as work progresses and reconcile all items before finishing.
- Mark each item as Done, Blocked (with a one-sentence reason and targeted question), or Cancelled.
- Avoid committing to tests or refactors unless you will do them now; otherwise label them as optional next steps.


# Special User Requests

- If a request can be satisfied directly via a terminal command (e.g. `date`), do so.
- If the user asks for a review, switch to review mode: prioritize bugs, risks, regressions, and missing coverage; present findings first, ordered by severity.
- If no issues are found, say so explicitly and note any remaining uncertainty or testing gaps.


# Frontend Tasks

- Avoid generic or interchangeable UI outcomes; aim for intentional, authored designs.
- Choose a clear visual direction in typography, color, motion, and layout.
- Use motion and background treatments deliberately, not as decoration.
- Ensure results are runnable and usable on both desktop and mobile within scope.
- When working inside an existing product or design system, preserve its established patterns and language.


# Presenting Your Work

- Produce plain text suitable for CLI rendering; add structure only where it improves scanability.
- Be concise and collaborative in tone.
- Explain what changed, where, and why; avoid dumping large files—reference paths instead.
- Suggest natural next steps only when they make sense.
- When asked for command output, summarize the important details rather than pasting raw output.


## Final Answer Structure and Style Guidelines

- Use headers sparingly; keep them short and meaningful.
- Prefer flat bullet lists with consistent phrasing.
- Use backticks for commands, paths, identifiers, and inline code.
- Wrap multi-line code in fenced blocks with language info where possible.
- Keep wording precise, active, and self-contained.
- For file references, use inline code paths and optional 1-based line numbers; avoid URIs and line ranges.

```
## Mid-Rollout User Updates

The Codex model family uses reasoning summaries to communicate user updates as it’s working. This can be in the form of one-liner headings (which updates the ephemeral text in Codex-CLI), or both heading and a short body. This is done by a separate model and therefore is **not promptable**, and we advise against adding any instructions to the prompt related to intermediate plans or messages to the user. We’ve improved these summaries for Codex-Max to be more communicative and provide more critical information about what’s happening and why; some of our users are updating their UX to promote these summaries more prominently in their UI, similar to how intermediate messages are displayed for GPT-5 series models.

## Using agents.md

Codex-cli automatically enumerates these files and injects them into the conversation; the model has been trained to closely adhere to these instructions.

1\. Files are pulled from \~/.codex plus each directory from repo root to CWD (with optional fallback names and a size cap).  
2\. They’re merged in order, later directories overriding earlier ones.  
3\. Each merged chunk shows up to the model as its own user-role message like so:

```
# AGENTS.md instructions for <directory>
<INSTRUCTIONS>
...file contents...
</INSTRUCTIONS>
```

Additional details

* Each discovered file becomes its own user-role message that starts with \# AGENTS.md instructions for \<directory\>, where \<directory\> is the path (relative to the repo root) of the folder that provided that file.  
* Messages are injected near the top of the conversation history, before the user prompt, in root-to-leaf order: global instructions first, then repo root, then each deeper directory. If an AGENTS.override.md was used, its directory name still appears in the header (e.g., \# AGENTS.md instructions for backend/api), so the context is obvious in the transcript.

# Compaction

Compaction unlocks significantly longer effective context windows, where user conversations can persist for many turns without hitting context window limits or long context performance degradation, and agents can perform very long trajectories that exceed a typical context window for long-running, complex tasks. A weaker version of this was previously possible with ad-hoc scaffolding and conversation summarization, but our first-class implementation, available via the Responses API, is integrated with the model and is highly performant.

How it works:

1. You use the Responses API as today, sending input items that include tool calls, user inputs, and assistant messages.  
2. When your context window grows large, you can invoke /compact to generate a new, compacted context window. Two things to note:  
   1. The context window that you send to /compact should fit within your model’s context window.  
   2. The endpoint is ZDR compatible and will return an “encrypted\_content” item that you can pass into future requests.  
3. For subsequent calls to the /responses endpoint, you can pass your updated, compacted list of conversation items (including the added compaction item). The model retains key prior state with fewer conversation tokens.

For endpoint details see our `/responses/compact` [docs](https://platform.openai.com/docs/api-reference/responses/compact).

# Tools

1. We strongly recommend using our exact `apply_patch` implementation as the model has been trained to excel at this diff format. For terminal commands we recommend our `shell` tool, and for plan/TODO items our `update_plan` tool should be most performant.  
2. If you prefer your agent to use more “terminal-like tools” (like `file_read()` instead of calling \`sed\` in the terminal), this model can reliably call them instead of terminal (following the instructions below)  
3. For other tools, including semantic search, MCPs, or other custom tools, they can work but it requires more tuning and experimentation.

### Apply\_patch

The easiest way to implement apply\_patch is with our first-class implementation in the Responses API, but you can also use our freeform tool implementation with [context-free grammar](https://cookbook.openai.com/examples/gpt-5/gpt-5_new_params_and_tools?utm_source=chatgpt.com#3-contextfree-grammar-cfg). Both are demonstrated below.

```py
# Sample script to demonstrate the server-defined apply_patch tool

import json
from pprint import pprint
from typing import cast

from openai import OpenAI
from openai.types.responses import ResponseInputParam, ToolParam

client = OpenAI()

## Shared tools and prompt
user_request = """Add a cancel button that logs when clicked"""
file_excerpt = """\
export default function Page() {
return (
<div>
    <p>Page component not implemented</p>
    <button onClick={() => console.log("clicked")}>Click me</button>
</div>
);
}
"""

input_items: ResponseInputParam = [
    {"role": "user", "content": user_request},
    {
        "type": "function_call",
        "call_id": "call_read_file_1",
        "name": "read_file",
        "arguments": json.dumps({"path": ("/app/page.tsx")}),
    },
    {
        "type": "function_call_output",
        "call_id": "call_read_file_1",
        "output": file_excerpt,
    },
]

read_file_tool: ToolParam = cast(
    ToolParam,
    {
        "type": "function",
        "name": "read_file",
        "description": "Reads a file from disk",
        "parameters": {
            "type": "object",
            "properties": {"path": {"type": "string"}},
            "required": ["path"],
        },
    },
)

### Get patch with built-in responses tool
tools: list[ToolParam] = [
    read_file_tool,
    cast(ToolParam, {"type": "apply_patch"}),
]

response = client.responses.create(
    model="gpt-5.1-Codex-Max",
    input=input_items,
    tools=tools,
    parallel_tool_calls=False,
)

for item in response.output:
    if item.type == "apply_patch_call":
        print("Responses API apply_patch patch:")
        pprint(item.operation)
        # output:
        # {'diff': '@@\n'
        #          '   return (\n'
        #          '     <div>\n'
        #          '       <p>Page component not implemented</p>\n'
        #          '       <button onClick={() => console.log("clicked")}>Click me</button>\n'
        #          '+      <button onClick={() => console.log("cancel clicked")}>Cancel</button>\n'
        #          '     </div>\n'
        #          '   );\n'
        #          ' }\n',
        #  'path': '/app/page.tsx',
        #  'type': 'update_file'}

### Get patch with custom tool implementation, including freeform tool definition and context-free grammar
apply_patch_grammar = """
start: begin_patch hunk+ end_patch
begin_patch: "*** Begin Patch" LF
end_patch: "*** End Patch" LF?

hunk: add_hunk | delete_hunk | update_hunk
add_hunk: "*** Add File: " filename LF add_line+
delete_hunk: "*** Delete File: " filename LF
update_hunk: "*** Update File: " filename LF change_move? change?

filename: /(.+)/
add_line: "+" /(.*)/ LF -> line

change_move: "*** Move to: " filename LF
change: (change_context | change_line)+ eof_line?
change_context: ("@@" | "@@ " /(.+)/) LF
change_line: ("+" | "-" | " ") /(.*)/ LF
eof_line: "*** End of File" LF

%import common.LF
"""

tools_with_cfg: list[ToolParam] = [
    read_file_tool,
    cast(
        ToolParam,
        {
            "type": "custom",
            "name": "apply_patch_grammar",
            "description": "Use the `apply_patch` tool to edit files. This is a FREEFORM tool, so do not wrap the patch in JSON.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": apply_patch_grammar,
            },
        },
    ),
]

response_cfg = client.responses.create(
    model="gpt-5.1-Codex-Max",
    input=input_items,
    tools=tools_with_cfg,
    parallel_tool_calls=False,
)

for item in response_cfg.output:
    if item.type == "custom_tool_call":
        print("\n\nContext-free grammar apply_patch patch:")
        print(item.input)
        #  Output
        # *** Begin Patch
        # *** Update File: /app/page.tsx
        # @@
        #      <div>
        #        <p>Page component not implemented</p>
        #        <button onClick={() => console.log("clicked")}>Click me</button>
        # +      <button onClick={() => console.log("cancel clicked")}>Cancel</button>
        #      </div>
        #    );
        #  }
        # *** End Patch
```

Patches objects the Responses API tool can be implemented by following this [example](https://github.com/openai/openai-agents-python/blob/main/examples/tools/apply_patch.py) and patches from the freeform tool can be applied with the logic in our canonical GPT-5 [apply\_patch.py](https://github.com/openai/openai-cookbook/blob/main/examples/gpt-5/apply_patch.py%20) implementation.

### Shell\_command

This is our default shell tool. Note that we have seen better performance with a command type “string” rather than a list of commands.

```
{
  "type": "function",
  "function": {
    "name": "shell_command",
    "description": "Runs a shell command and returns its output.\n- Always set the `workdir` param when using the shell_command function. Do not use `cd` unless absolutely necessary.",
    "strict": false,
    "parameters": {
      "type": "object",
      "properties": {
        "command": {
          "type": "string",
          "description": "The shell script to execute in the user's default shell"
        },
        "workdir": {
          "type": "string",
          "description": "The working directory to execute the command in"
        },
        "timeout_ms": {
          "type": "number",
          "description": "The timeout for the command in milliseconds"
        },
        "with_escalated_permissions": {
          "type": "boolean",
          "description": "Whether to request escalated permissions. Set to true if command needs to be run without sandbox restrictions"
        },
        "justification": {
          "type": "string",
          "description": "Only set if with_escalated_permissions is true. 1-sentence explanation of why we want to run this command."
        }
      },
      "required": ["command"],
      "additionalProperties": false
    }
  }
}
```

If you’re using Windows PowerShell, update to this tool description.

```
Runs a shell command and returns its output. The arguments you pass will be invoked via PowerShell (e.g., ["pwsh", "-NoLogo", "-NoProfile", "-Command", "<cmd>"]). Always fill in workdir; avoid using cd in the command string.
```

You can check out codex-cli for the implementation for `exec_command`, which launches a long-lived PTY when you need streaming output, REPLs, or interactive sessions; and `write_stdin`, to feed extra keystrokes (or just poll output) for an existing exec\_command session.

### Update Plan

This is our default TODO tool; feel free to customize as you’d prefer. See the `## Plan tool` section of our starter prompt for additional instructions to maintain hygiene and tweak behavior.

```json
{
  "type": "function",
  "function": {
    "name": "update_plan",
    "description": "Updates the task plan.\nProvide an optional explanation and a list of plan items, each with a step and status.\nAt most one step can be in_progress at a time.",
    "strict": false,
    "parameters": {
      "type": "object",
      "properties": {
        "explanation": {
          "type": "string"
        },
        "plan": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "step": {
                "type": "string"
              },
              "status": {
                "type": "string",
                "description": "One of: pending, in_progress, completed"
              }
            },
            "additionalProperties": false,
            "required": [
              "step",
              "status"
            ]
          },
          "description": "The list of steps"
        }
      },
      "additionalProperties": false,
      "required": [
        "plan"
      ]
    }
  }
}
```

### View\_image

This is a basic function used in codex-cli for the model to view images.

```
{
  "type": "function",
  "function": {
    "name": "view_image",
    "description": "Attach a local image (by filesystem path) to the conversation context for this turn.",
    "strict": false,
    "parameters": {
      "type": "object",
      "properties": {
        "path": {
          "type": "string",
          "description": "Local filesystem path to an image file"
        }
      },
      "additionalProperties": false,
      "required": [
        "path"
      ]
    }
  }
}


```

## Dedicated terminal-wrapping tools

If you would prefer your codex agent to use terminal-wrapping tools (like a dedicated `list_dir(‘.’)` tool instead of `terminal(‘ls .’)`, this generally works well. We see the best results when the name of the tool, the arguments, and the output are as close as possible to those from the underlying command, so it’s as in-distribution as possible for the model (which was primarily trained using a dedicated terminal tool). For example, if you notice the model using git via the terminal and would prefer it to use a dedicated tool, we found that creating a related tool, and adding a directive in the prompt to only use that tool for git commands, fully mitigated the model’s terminal usage for git commands.

```
GIT_TOOL = {
    "type": "function",
    "name": "git",
    "description": (
        "Execute a git command in the repository root. Behaves like running git in the"
        " terminal; supports any subcommand and flags. The command can be provided as a"
        " full git invocation (e.g., `git status -sb`) or just the arguments after git"
        " (e.g., `status -sb`)."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "command": {
                "type": "string",
                "description": (
                    "The git command to execute. Accepts either a full git invocation or"
                    " only the subcommand/args."
                ),
            },
            "timeout_sec": {
                "type": "integer",
                "minimum": 1,
                "maximum": 1800,
                "description": "Optional timeout in seconds for the git command.",
            },
        },
        "required": ["command"],
    },
}

...

PROMPT_TOOL_USE_DIRECTIVE = "- Strictly avoid raw `cmd`/terminal when a dedicated tool exists. Default to solver tools: `git` (all git), `list_dir`, `apply_patch`. Use `cmd`/`run_terminal_cmd` only when no listed tool can perform the action." # update with your desired tools
```

## Other Custom Tools (web search, semantic search, memory, etc.)

The model hasn’t necessarily been post-trained to excel at these tools, but we have seen success here as well. To get the most out of these tools, we recommend:

1. Making the tool names and arguments as semantically “correct” as possible, for example “search” is ambiguous but “semantic\_search” clearly indicates what the tool does, relative to other potential search-related tools you might have. “Query” would be a good param name for this tool.  
2. Be explicit in your prompt about when, why, and how to use these tools, including good and bad examples.  
3. It could also be helpful to make the results look different from outputs the model is accustomed to seeing from other tools, for example ripgrep results should look different from semantic search results to avoid the model collapsing into old habits.

## Parallel Tool Calling

In codex-cli, when parallel tool calling is enabled, the responses API request sets `parallel_tool_calls: true` and the following snippet is added to the system instructions:

```
## Exploration and reading files

- **Think first.** Before any tool call, decide ALL files/resources you will need.
- **Batch everything.** If you need multiple files (even from different places), read them together.
- **multi_tool_use.parallel** Use `multi_tool_use.parallel` to parallelize tool calls and only this.
- **Only make sequential calls if you truly cannot know the next file without seeing a result first.**
- **Workflow:** (a) plan all needed reads → (b) issue one parallel batch → (c) analyze results → (d) repeat if new, unpredictable reads arise.

**Additional notes**:
- Always maximize parallelism. Never read files one-by-one unless logically unavoidable.
- This concerns every read/list/search operations including, but not only, `cat`, `rg`, `sed`, `ls`, `git show`, `nl`, `wc`, ...
- Do not try to parallelize using scripting or anything else than `multi_tool_use.parallel`.
```

We've found it to be helpful and more in-distribution if parallel tool call items and responses are ordered in the following way:

```
function_call
function_call
function_call_output
function_call_output
```

## Tool Response Truncation

We recommend doing tool call response truncation as follows to be as in-distribution for the model as possible:

* Limit to 10k tokens. You can cheaply approximate this by computing `num_bytes/4`.  
* If you hit the truncation limit, you should use half of the budget for the beginning, half for the end, and truncate in the middle with `…3 tokens truncated…`
